# Data Exploration

### Imports

In [ ]:
# Imports
from datetime import datetime
from bs4 import BeautifulSoup

import re
import requests, datetime
import pandas as pd
import numpy as np
import pprint
from urllib.request import Request, urlopen
from html.parser import HTMLParser

from multiprocessing.pool import ThreadPool

print("--- Complete ---")

--- Complete ---


## Crunchbase Company Data

In [ ]:
# Queries limited to 1000 rows each so they are saved/loaded in separately
q1_df = pd.read_csv('crunchbase_query1.csv')
q2_df = pd.read_csv('crunchbase_query2.csv')
q3_df = pd.read_csv('crunchbase_query3.csv')

In [ ]:
# Join the two dataframes together
merged_df = pd.concat([q1_df, q2_df, q3_df])

In [ ]:
# Drop duplicates, using two columns to be more certain of duplciates
df = merged_df.drop_duplicates(subset=['Organization Name', 'Organization Name URL']).reset_index(drop=True)

In [ ]:
df.head()

,Organization Name,Organization Name URL,Industries,Headquarters Location,Description,CB Rank (Company),Number of Funding Rounds,IPqwery - Patents Pending,IPqwery - Patents Granted,IPqwery - Total Patents,Founded Date,Founded Date Precision,Number of Articles,Full Description,Company Type,Website,Industry Groups,Founders,Top 5 Investors,Last Leadership Hiring Date,Last Layoff Mention Date,Headquarters Regions,Diversity Spotlight (US Only),Estimated Revenue Range,Operating Status,Exit Date,Exit Date Precision,Closed Date,Closed Date Precision,Twitter,Facebook,LinkedIn,Contact Email,Phone Number,Hub Tags,Investor Type,Investment Stage,Number of Portfolio Organizations,Number of Investments,Number of Lead Investments,...,BuiltWith - Tech Changes (%),BuiltWith - Active Tech Count,Apptopia - Number of Apps,Apptopia - Downloads Last 30 Days,G2 Stack - Total Products Active,G2 Stack - Total Product Changes,IPqwery - IP Activity Score,IPqwery - Trademarks Pending,IPqwery - Trademarks Registered,IPqwery - Total Trademarks,IPqwery - Most Popular Patent Class,IPqwery - Most Popular Class Description,IPqwery - Most Popular Trademark Class,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),Aberdeen - Software Spend,Aberdeen - Software Spend Currency,Aberdeen - Software Spend Currency (in USD),Aberdeen - Communications Spend,Aberdeen - Communications Spend Currency,Aberdeen - Communications Spend Currency (in USD),Aberdeen - Services Spend,Aberdeen - Services Spend Currency,Aberdeen - Services Spend Currency (in USD),Aberdeen - Server Spend,Aberdeen - Server Spend Currency,Aberdeen - Server Spend Currency (in USD),Aberdeen - PC Spend,Aberdeen - PC Spend Currency,Aberdeen - PC Spend Currency (in USD),Aberdeen - Storage Spend,Aberdeen - Storage Spend Currency,Aberdeen - Storage Spend Currency (in USD),Aberdeen - Other Hardware Spend,Aberdeen - Other Hardware Spend Currency,Aberdeen - Other Hardware Spend Currency (in USD),Aberdeen - Other IT Spend,Aberdeen - Other IT Spend Currency,Aberdeen - Other IT Spend Currency (in USD)
0,100 Percent Pure,https://www.crunchbase.com/organization/100-pe...,"Beauty, Solar","San Jose, California, United States","The healthiest, the most pure skincare in the ...","132,235",NaN,NaN,NaN,NaN,2004-01-01,year,1,"The absolute most healthy, the most pure line ...",For Profit,http://www.100percentpure.com/,"Consumer Goods, Energy, Natural Resources, Sus...",Susie Wang,NaN,NaN,NaN,"San Francisco Bay Area, Silicon Valley, West C...",NaN,$50M to $100M,Active,NaN,NaN,NaN,NaN,https://twitter.com/@100percentpure,https://www.facebook.com/100percentpure,https://www.linkedin.com/company/purity-cosmetics,customerservice@puritycosmetics.com,415-814-9788,NaN,NaN,NaN,NaN,NaN,NaN,...,0%,21.0,NaN,NaN,45.0,7.14%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,180 Smoke Vape Wholesale,https://www.crunchbase.com/organization/180-sm...,"E-Commerce, Retail, Wholesale","Toronto, Ontario, Canada",Canada's leading e-cigarette company for retai...,"191,231",NaN,NaN,NaN,NaN,2012-01-01,year,NaN,NaN,For Profit,http://www.180smoke.ca/vape-wholesale,Commerce and Shopping,NaN,NaN,NaN,NaN,Great Lakes,NaN,Less than $1M,Active,NaN,NaN,NaN,NaN,https://www.twitter.com/180smoke,https://www.facebook.com/180smoke,NaN,NaN,+1 (855) 994-6180,NaN,NaN,NaN,NaN,NaN,NaN,...,0%,59.0,NaN,NaN,24.0,-4%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25180.0,USD,25180.0,3564.0,USD,3564.0,820.0,USD,820.0,13059.0,USD,13059.0,NaN,USD,NaN,1015.0,USD,1015.0,96.0,USD,96.0,693.0,USD,693.0,5933.0,USD,5933.0
2,22nd Century Group,https://www.crunchbase.com/organization/22nd-c...,"Biotechnology, Health Care, Health Diagnostics","Clarence, New York, United States",22nd Century Group is a plant biotechnology co...,"30,219",6.0,NaN,NaN,NaN,1998-01-01,year,63,22nd Century Group is a plant biotechnology co...,For Profit,http://www.xxiicentury.com,"Biotechnology, Health Care, Sci

In [ ]:
# Number of companies returned by the query
len(df)

1556

In [ ]:
# There are lots of null values
df.isna().sum()

Organization Name                                       0
Organization Name URL                                   0
Industries                                             48
Headquarters Location                                 235
Description                                             0
                                                     ... 
Aberdeen - Other Hardware Spend Currency             1360
Aberdeen - Other Hardware Spend Currency (in USD)    1365
Aberdeen - Other IT Spend                            1406
Aberdeen - Other IT Spend Currency                   1360
Aberdeen - Other IT Spend Currency (in USD)          1406
Length: 149, dtype: int64

In [ ]:
# The data certainly needs cleaning and refining,
# e.g. the following entry is included due to the 'smoking' search term
df[df['Organization Name'] == 'Farmland Foods']['Full Description'].reset_index(drop=True)[0]

"They love hand-trimming each delicious pork chop and juicy tenderloin. They love slow smoking every strip of their signature bacon with real hardwood hickory chips. And they love finding new ways to enjoy pork, so you can too. Simply put, they have passion for pork. They've worked with local farm families for over 50 years to secure the best quality pork, and no one takes more time, effort or care in providing delicious, fresh pork and bacon. Breakfast, lunch or dinner, it's a difference you can taste in every bite."

## Job Search Data

The following functions allow you to scrape job data from Indeed US. 

### Job Scraper Class

In [ ]:
class JobPostingScraper:
  """
  A scraper for Indeed US that populates job postings into a DataFrame.
  """

  def create_job_urls(self, companies) -> list:
  '''
  Generates the URLS for each search term given
  '''
    url_list = []
    for company in companies:
      company = company.replace(" ", "+")
      url = "https://www.indeed.com/jobs?q={}&l=".format(company)
      url_list.append(url)
    return url_list

    def scrape_data(self, url_to_scrape: str, number_of_pages: int) -> list:
      '''
      Scrapes data from Indeed page.
      '''
      data_collected = []
      for i in range(0, number_of_pages):
        extension = ''
        if i is not 0:
          extension = "&start".format(i*10)
        url = url_to_scrape + extension
        req = Request(url)
        page = urlopen(req).read()
        soup = BeautifulSoup(page, 'html.parser')
        data_collected = get_data_from_page(data_collected, soup)
      return data_collected, soup

    def get_data_from_page(self, data_collected: list, soup) -> list:
      '''
      Collects all page information depending on number of pages selected 
      '''
      job_posts = []
      for div in soup.findAll('div', attrs={'class': 'jobsearch-SerpJobCard unifiedRow row result'}):
        job = dict()
        job = extract_data_points(job, div)
        job_posts.append(div['data-jk'])
        single_job_post_extension_url = "https://www.indeed.com/viewjob?jk=" + div['data-jk']
        job['url'] = single_job_post_extension_url
        req = Request(single_job_post_extension_url)
        job_page = urlopen(req).read()
        # job_page = requests.get(single_job_post_extension_url)
        job_soup = BeautifulSoup(job_page, 'html.parser')

        for inside_div in job_soup.findAll('div', attrs={"class": "jobsearch-jobDescriptionText"}):
          details = inside_div.text.strip() + "..." # change for longer description
          job['details'] = re.sub(' +', ' ', details.replace("\n", " "))
        data_collected.append(job)
      return data_collected


      def extract_data_points(self, job, div) -> dict:
        '''
        Searches for specific information on page
        '''
        for a in div.findAll(name='a', attrs={'data-tn-element': 'jobTitle'}):
          job['title'] = a['title']
        for a1 in div.findAll('a', attrs={'data-tn-element': 'companyName'}):
          job['companyName'] = a1.text.strip()
        for span in div.findAll('span', attrs={'class': 'ratingsContent'}):
          job['rating'] = span.text.strip()
        for span1 in div.findAll('span', attrs={'class': 'location accessible-contrast-color-location'}):
          job['location'] = span1.text.strip()
        for div1 in div.findAll('div', attrs={'class': 'summary'}):
          summary = div1.text.strip()
          job['summary'] = summary
          # job['summary'] = re.sub(' +', ' ', summary.replace("\n", ""))
        for span2 in div.findAll('span', attrs={'class': 'date'}):
          job['date'] = span2.text.strip()
        return job

      def create_job_dataset(self, url_list):
        '''
        Returns each company search into list.
        '''
        jobs_data = []
        for url in url_list:
          jobs = scrape_data(url, 3)
          jobs_data.append(jobs)
        return jobs_data
      
      def join_job_dataset(self, jobs_list) -> DataFrame:
        '''
        Takes nested dictionaries and concatenates into one DataFrame object
        '''
        jobs_df = pd.DataFrame()
        for data in jobs_list:
          jobs_df = pd.concat([jobs_df, pd.DataFrame(data)], ignore_index=True)
        return jobs_df

### Job Scraping Functions

In [ ]:
def create_job_urls(companies):
  '''
  Generates the URLS for each search term given
  '''
  url_list = []
  for company in companies:
    company = company.replace(" ", "+")

    url = "https://www.indeed.com/jobs?q={}&l=".format(company)
    url_list.append(url)
  return url_list

In [ ]:
def get_data_from_page(data_collected, soup):
  '''
  Collects all page information depending on number of pages selected 
  '''
  job_posts = []
  for div in soup.findAll('div', attrs={'class': 'jobsearch-SerpJobCard unifiedRow row result'}):
    job = dict()
    job = extract_data_points(job, div)
    job_posts.append(div['data-jk'])
    single_job_post_extension_url = "https://www.indeed.com/viewjob?jk=" + div['data-jk']
    job['url'] = single_job_post_extension_url
    req = Request(single_job_post_extension_url)
    job_page = urlopen(req).read()
    # job_page = requests.get(single_job_post_extension_url)
    job_soup = BeautifulSoup(job_page, 'html.parser')

    for inside_div in job_soup.findAll('div', attrs={"class": "jobsearch-jobDescriptionText"}):
      details = inside_div.text.strip() + "..." # change for longer description
      job['details'] = re.sub(' +', ' ', details.replace("\n", " "))
    data_collected.append(job)
  return data_collected


In [ ]:
def scrape_data(url_to_scrape, number_of_pages):
  '''
  Scrapes data from Indeed page.
  '''
  data_collected = []
  for i in range(0, number_of_pages):
    extension = ''
    if i is not 0:
      extension = "&start".format(i*10)
    url = url_to_scrape + extension
    req = Request(url)
    page = urlopen(req).read()
    # page = requests.get(url)
    soup = BeautifulSoup(page, 'html.parser')
    data_collected = get_data_from_page(data_collected, soup)
  return data_collected

In [ ]:
def extract_data_points(job, div):
  '''
  Searches for specific information on page
  '''
  for a in div.findAll(name='a', attrs={'data-tn-element': 'jobTitle'}):
    job['title'] = a['title']
  for a1 in div.findAll('a', attrs={'data-tn-element': 'companyName'}):
    job['companyName'] = a1.text.strip()
  for span in div.findAll('span', attrs={'class': 'ratingsContent'}):
    job['rating'] = span.text.strip()
  for span1 in div.findAll('span', attrs={'class': 'location accessible-contrast-color-location'}):
    job['location'] = span1.text.strip()
  for div1 in div.findAll('div', attrs={'class': 'summary'}):
    summary = div1.text.strip()
    job['summary'] = summary
    # job['summary'] = re.sub(' +', ' ', summary.replace("\n", ""))
  for span2 in div.findAll('span', attrs={'class': 'date'}):
    job['date'] = span2.text.strip()
  return job

In [ ]:
def create_job_dataset(url_list):
  '''
  Returns each company search into list.
  '''
  jobs_data = []
  for url in url_list:
    jobs = scrape_data(url, 3)
    jobs_data.append(jobs)
  return jobs_data

In [ ]:
def join_job_dataset(jobs_list):
  '''
  Takes nested dictionaries and concatenates into one DataFrame object
  '''
  jobs_df = pd.DataFrame()
  for data in jobs_list:
    jobs_df = pd.concat([jobs_df, pd.DataFrame(data)], ignore_index=True)
  return jobs_df

### Job Scraping

Defining the names of companies to search

In [ ]:
companies_df = pd.read_excel("/content/Competitive Innovation - Data Exploration Sources - 210225.xlsx", 1)

In [ ]:
companies_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Competitive Innovation: Jobs & Patents Company...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,For data gather,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Company,Patents Assignee Input,Indeed Company Name Input,Justification,Column1,NaN,NaN,Comments
4,Reynolds American,Reynolds Tobacco OR RAI strategic holdings,https://www.indeed.com/jobs?q=Reynolds+America...,Tobacco Industry Player,James H,NaN,NaN,NaN


In [ ]:
companies_df = companies_df.iloc[3:]

In [ ]:
new_header = companies_df.iloc[0] #grab the first row for the header
companies_df = companies_df[1:] #take the data less the header row
companies_df.columns = new_header #set the header row as the df header

In [ ]:
companies_df.head(10)

3,Company,Patents Assignee Input,Indeed Company Name Input,Justification,Column1,NaN,NaN,Comments
4,Reynolds American,Reynolds Tobacco OR RAI strategic holdings,https://www.indeed.com/jobs?q=Reynolds+America...,Tobacco Industry Player,James H,NaN,NaN,NaN
5,British American Tobacco,British American Tobacco OR nicoventures,https://www.indeed.com/jobs?q=British+American...,Tobacco Industry Player,James H,NaN,NaN,NaN
6,Philip Morris,Philip Morris,https://www.indeed.com/jobs?q=Philip+Morris&l=,Tobacco Industry Player,James H,NaN,NaN,NaN
7,Altria,Altria,https://www.indeed.com/jobs?q=Altria&l=,Tobacco Industry Player,James H,NaN,NaN,NaN
8,Imperial Tobacco,Imperial Tobacco,https://www.indeed.com/jobs?q=Imperial+Tobacco&l=,Tobacco Industry Player,James H,NaN,NaN,NaN
9,Japan Tobacco,Japan Tobacco,https://www.indeed.com/jobs?q=Japan+Tobacco&l=,Tobacco Industry Player,James H,NaN,NaN,NaN
10,KT&G,KT&G,https://www.indeed.com/jobs?q=KT&G&l=,Tobacco Industry Player,James H,NaN,NaN,NaN
11,Lorillard,Lorillard,https://www.indeed.com/jobs?q=Lorillard&l=,Tobacco Industry Player,James C,NaN,NaN,NaN
12,Juul,Juul,https://www.indeed.com/jobs?q=Juul&l=,Vapour Industry Player,James C,NaN,NaN,NaN
13,Njoy,Njoy,https://www.indeed.com/jobs?q=Njoy&l=,Vapour Industry Player,James C,NaN,NaN,NaN


In [ ]:
# using the companies data, generate search URLs
companies = companies_df['Company'].tolist()
url_list = create_job_urls(companies)
pprint.pprint(url_list)

NameError: ignored

In [ ]:
companies_df['Indeed Company Name Input'] = pd.Series(url_list, index=companies_df.index)

In [ ]:
companies_df

3,Company,Patents Assignee Input,Indeed Company Name Input,Justification,url
4,Reynolds American,Reynolds American,https://www.indeed.com/jobs?q=Reynolds+America...,Tobacco Industry Player,https://www.indeed.com/jobs?q=Reynolds+America...
5,British American Tobacco,British American Tobacco,https://www.indeed.com/jobs?q=British+American...,Tobacco Industry Player,https://www.indeed.com/jobs?q=British+American...
6,Nicoventures,Nicoventures,https://www.indeed.com/jobs?q=Nicoventures&l=,Tobacco Industry Player,https://www.indeed.com/jobs?q=Nicoventures&l=
7,Philip Morris,Philip Morris,https://www.indeed.com/jobs?q=Philip+Morris&l=,Tobacco Industry Player,https://www.indeed.com/jobs?q=Philip+Morris&l=
8,Altria,Altria,https://www.indeed.com/jobs?q=Altria&l=,Tobacco Industry Player,https://www.indeed.com/jobs?q=Altria&l=
9,Imperial Tobacco,Imperial Tobacco,https://www.indeed.com/jobs?q=Imperial+Tobacco&l=,Tobacco Industry Player,https://www.indeed.com/jobs?q=Imperial+Tobacco&l=
10,Japan Tobacco,Japan Tobacco,https://www.indeed.com/jobs?q=Japan+Tobacco&l=,Tobacco Industry Player,https://www.indeed.com/jobs?q=Japan+Tobacco&l=
11,KT&G,KT&G,https://www.indeed.com/jobs?q=KT&G&l=,Tobacco Industry Player,https://www.indeed.com/jobs?q=KT&G&l=
12,Lorillard,Lorillard,https://www.indeed.com/jobs?q=Lorillard&l=,Tobacco Industry Player,https://www.indeed.com/jobs?q=Lorillard&l=
13,Juul,Juul,https://www.indeed.com/jobs?q=Juul&l=,Vapour Industry Player,https://www.indeed.com/jobs?q=Juul&l=


In [ ]:
companies_df.to_csv("companies data.csv", index=False)

In [ ]:
# create list of jobs data for specified companies
jobs_data = create_job_dataset(url_list[5:])
print(jobs_data)

[[{'title': 'Retail Development Representative - Coventry/Solihull', 'companyName': 'Imperial Brands', 'rating': '4.1', 'location': 'United States', 'summary': 'Drive customer advocacy for our brands through educating retailers in the tobacco and NGP market and industry legislation.', 'date': '21 days ago', 'url': 'https://www.indeed.com/viewjob?jk=7c486c0bfbbd15b7', 'details': "About us At Imperial Brands, we’re challengers by instinct and energetic hands-on operators. Our entrepreneurial spirit is what sets us apart. We’re a truly international company driven by a strong challenger culture. An innovative and responsible FTSE100 FMCG business widely recognised on the global stage, we generate over £30bn revenue and employ over 27,500 like-minded achievers in markets worldwide. We’re driven by a common purpose: to create something better for the world’s smokers including next generation product consumers. Alongside our respected brands such as Lambert & Butler, John Player Special and 

In [ ]:
# join the lsit of jobs data together
jobs_df = join_job_dataset(jobs_data)
jobs_df

,title,companyName,rating,location,summary,date,url,details
0,Retail Development Representative - Coventry/S...,Imperial Brands,4.1,United States,Drive customer advocacy for our brands through...,21 days ago,https://www.indeed.com/viewjob?jk=7c486c0bfbbd...,"About us At Imperial Brands, we’re challengers..."
1,Retail Development Representative - London,Imperial Brands,4.1,United States,Drive customer advocacy for our brands through...,9 days ago,https://www.indeed.com/viewjob?jk=9ff52abd4fee...,"About us At Imperial Brands, we’re challengers..."
2,Staff,NaN,NaN,"Sullivan, MO 63080","Adhere to all the city, county and state liquo...",30+ days ago,https://www.indeed.com/viewjob?jk=b2393b4d20f6...,PURPOSE: Represent and promote the store opera...
3,2nd Shift Warehouse Selector,Gordon Food Service,3.6,"Imperial, PA 15126",2nd Shift Warehouse Selectors start at $17.64/...,14 days ago,https://www.indeed.com/viewjob?jk=276b3edcc24c...,Header: 2nd Shift Warehouse Selectors start at...
4,Divisional Transportation Router,Gordon Food Service,3.6,"Imperial, PA 15126",Our Division Router here at Gordon Food Servic...,13 days ago,https://www.indeed.com/viewjob?jk=04b26fa1339a...,Header: Welcome to Gordon Food Service! We are...
...,...,...,...,...,...,...,...,...
325,Health Program Assistant 2 (35 Hour),State of Connecticut - Department of Public He...,4.0,"Hartford, CT",This Health Program Assistant 2 position will ...,3 days ago,https://www.indeed.com/viewjob?jk=e04530524628...,"The State of Connecticut, Department of Public..."
326,Plant Lab Technician - 2nd Shift (1pm-9:30pm),Benjamin Moore,3.6,"Newark, NJ 07105",Follows all safety guidelines and rules; envir...,30+ days ago,https://www.indeed.com/viewjob?jk=cf173c2c21ef...,Job Summary: Follows all safety guidelines and...
327,Warehouse Associate - 3rd Shift,Benjamin Moore,3.6,"Clifton, NJ 07011","We offer excellent compensation, earn up to $2...",30+ days ago,https://www.indeed.com/viewjob?jk=0ed14ef2833a...,Warehouse Associate Employment Highlights: We ...
328,Territory Representative - Denver,Benjamin Moore,3.6,Illinois,You will be accountable for delivering sustain...,2 days ago,https://www.indeed.com/viewjob?jk=bcf72aff65fd...,Territory Representative – Denver Benjamin Moo...


In [ ]:
jobs_df['details'].iloc[11]

"If working for a company that values its people and recognizes the importance of work/life balance is appealing to you, look no further than ITG Brands! WHO WE ARE ITG Brands is the U.S. division of Imperial Brands. We are the nation's third-largest tobacco company with offerings of some of the most well-known cigarette, cigar, and vaping products. ITG Brands carries at its core, the mindset that, in order to succeed, a company must treat its people well and give them the tools they need to win. We are currently searching for a Supervisor, Production Maintenance for our Operations facility located in Greensboro, North Carolina. In this role, you will work with and report directly to the Superintendent of Maintenance and will be responsible for planing and supervising the activities of hourly maintenance employees, outside contractors, and skilled labor in the maintenance of production systems, utilities, and/or buildings and grounds. Supervise skilled labor, outside contractors, and s

In [ ]:
# check individual url
jobs_df['url'].iloc[1]

'https://www.indeed.com/viewjob?jk=9ff52abd4fee18f0'

In [ ]:
# export data set to csv
jobs_df.to_csv("jobs_data1.csv",index=False)

## Patent Data from Google Patents

### Patent Data Cleaning

In [ ]:
patent_df = pd.read_csv('gp-search-test.csv')

In [ ]:
# Need to do some cleaning
patent_df.head(2)

,,,,,,,,search URL:,https://patents.google.com/?q=vape&after=priority:20160101&sort=new
id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
AU-2020294244-A1,Modular thermal storage,Malta Inc.,"Raj Apte, Julian Green, Philippe Larochelle",2016-12-31,2020-12-23,2021-01-28,NaN,https://patents.google.com/patent/AU2020294244...,NaN


In [ ]:
# First reset the index
patent_df = patent_df.reset_index()

In [ ]:
patent_df.head(2)

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,search URL:,https://patents.google.com/?q=vape&after=priority:20160101&sort=new
0,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
1,AU-2020294244-A1,Modular thermal storage,Malta Inc.,"Raj Apte, Julian Green, Philippe Larochelle",2016-12-31,2020-12-23,2021-01-28,NaN,https://patents.google.com/patent/AU2020294244...,NaN


In [ ]:
# The first row should be the headers, lets fix that
# Get the first row
new_header = patent_df.iloc[0]
# Get the data, excluding the first row
patent_df = patent_df[1:] 
# Set the column names 
patent_df.columns = new_header
# Reset the index to start from 0
patent_df = patent_df.reset_index(drop=True)

In [ ]:
patent_df.head(5)

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,AU-2020294244-A1,Modular thermal storage,Malta Inc.,"Raj Apte, Julian Green, Philippe Larochelle",2016-12-31,2020-12-23,2021-01-28,NaN,https://patents.google.com/patent/AU2020294244...,NaN
1,AU-2020281064-A1,Bicyclic compounds,Basf Se,"Ashokkumar Adisechan, Rupsha Chaudhuri, Gopal ...",2016-04-01,2020-12-02,2021-01-07,NaN,https://patents.google.com/patent/AU2020281064...,NaN
2,AU-2020273366-A1,Variable pressure inventory control of closed ...,Malta Inc.,"Raj Apte, Philippe Larochelle",2016-12-28,2020-11-20,2020-12-17,NaN,https://patents.google.com/patent/AU2020273366...,NaN
3,AU-2020267295-A1,Use of external air for closed cycle inventory...,Malta Inc.,"Raj Apte, Philippe Larochelle",2016-12-29,2020-11-13,2020-12-10,NaN,https://patents.google.com/patent/AU2020267295...,NaN
4,AU-2020267186-A1,Methods for photodynamic therapy,"Dusa Pharmaceuticals, Inc.","Michael Guttadauro, Scott Lundahl",2018-01-12,2020-11-10,2020-12-03,NaN,https://patents.google.com/patent/AU2020267186...,NaN


### Patent Abstracts

In [ ]:
startTime = datetime.datetime.now()

In [ ]:
first_5_links = patent_df['result link'][:5]
first_5_links

0    https://patents.google.com/patent/AU2020294244...
1    https://patents.google.com/patent/AU2020281064...
2    https://patents.google.com/patent/AU2020273366...
3    https://patents.google.com/patent/AU2020267295...
4    https://patents.google.com/patent/AU2020267186...
Name: result link, dtype: object

In [ ]:
abstracts = []
print("Starting loop at " + str(startTime)) 

for link in patent_df['result link'][:10]:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    text = soup.find(name='abstract').prettify()
    abstracts.append(text)

print("It took {} to run this script".format(datetime.datetime.now() - startTime))        


Starting loop at 2021-03-08 16:53:32.098343
It took 0:00:13.620339 to run this script


Multithreading

In [ ]:
def get_abstract(url):
  """
  Given a URL, returns the Google Patents abstract.
  """
  try:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    text = soup.find(name='abstract').prettify()
    return text
  except:
    return "error"


In [ ]:
# abstracts_dict = {'Abstract':[]}
# df_abstract = pd.DataFrame.from_dict(abstracts_dict)
# df_abstract.head()

,Abstract


In [ ]:
abstracts_multi = []

print("Starting loop at " + str(startTime)) 

with ThreadPool(10) as pool:
  for result in pool.map(get_abstract, patent_df['result link'][:1000]):
    #df_abstract = df_abstract.append(pd.Series(result), ignore_index=True, sort=False)
    abstracts_multi.append(result)

print("It took {} to run this script".format(datetime.datetime.now() - startTime))        


Starting loop at 2021-03-08 16:53:32.098343
It took 0:16:58.906878 to run this script


In [ ]:
abstracts_multi

['<abstract lang="EN" load-source="patent-office" mxw-id="PA431671545">\n <div class="abstract" id="p0001" num="1">\n  MODULAR THERMAL STORAGE \nABSTRACT \nA power generation system comprising a shared hot side thermal store, a shared cold \nside thermal store, a plurality of power subunits, and an electrical bus is disclosed. Each of the \npower subunits may connected or isolated from the shared hot side thermal store and/or the \nshared cold side thermal store.\n </div>\n</abstract>\n',
 '<abstract lang="EN" load-source="patent-office" mxw-id="PA428459510">\n <div class="abstract" id="p0001" num="1">\n  Bicyclic compounds \nAbstract \nThe present invention relates to compounds of formula I, \n5 \n2 \nR \nR N \nPONQ&gt; -Ar \n(I) \nwherein the variables are defined as given in the description and claims. The invention further \n0 relates to uses, processes and composition for compounds I.  \n16902155_1 (GHMatters)P44390AU01\n </div>\n</abstract>\n',
 '<abstract lang="EN" load-source="

In [ ]:
patent_df['Abstract'] = pd.Series(abstracts_multi)

In [ ]:
patent_df.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,Abstract
0,AU-2020294244-A1,Modular thermal storage,Malta Inc.,"Raj Apte, Julian Green, Philippe Larochelle",2016-12-31,2020-12-23,2021-01-28,NaN,https://patents.google.com/patent/AU2020294244...,NaN,"<abstract lang=""EN"" load-source=""patent-office..."
1,AU-2020281064-A1,Bicyclic compounds,Basf Se,"Ashokkumar Adisechan, Rupsha Chaudhuri, Gopal ...",2016-04-01,2020-12-02,2021-01-07,NaN,https://patents.google.com/patent/AU2020281064...,NaN,"<abstract lang=""EN"" load-source=""patent-office..."
2,AU-2020273366-A1,Variable pressure inventory control of closed ...,Malta Inc.,"Raj Apte, Philippe Larochelle",2016-12-28,2020-11-20,2020-12-17,NaN,https://patents.google.com/patent/AU2020273366...,NaN,"<abstract lang=""EN"" load-source=""patent-office..."
3,AU-2020267295-A1,Use of external air for closed cycle inventory...,Malta Inc.,"Raj Apte, Philippe Larochelle",2016-12-29,2020-11-13,2020-12-10,NaN,https://patents.google.com/patent/AU2020267295...,NaN,"<abstract lang=""EN"" load-source=""patent-office..."
4,AU-2020267186-A1,Methods for photodynamic therapy,"Dusa Pharmaceuticals, Inc.","Michael Guttadauro, Scott Lundahl",2018-01-12,2020-11-10,2020-12-03,NaN,https://patents.google.com/patent/AU2020267186...,NaN,"<abstract lang=""EN"" load-source=""patent-office..."


In [ ]:
test_url = "https://patents.google.com/patent/US9215895B2/en?q=nicotine&country=US"

### Patents Classifications

In [ ]:
startTime = datetime.datetime.now()

In [ ]:
# Creating a function to grab patent classifications and return an output dictionary

def patent_classifications_scraper(soup: BeautifulSoup) -> dict:
  '''
  Function to grab patent classifications and return output dictionary.
  Must pass a soup object. 
  '''
  output = {}
  try:
    for li_tag in soup.find_all("ul", itemprop = "cpcs"):
      for span_tag in li_tag.find_all('li', itemprop= "cpcs"):
        field = span_tag.find('span', itemprop="Code").text
        value = span_tag.find('span', itemprop="Description").text
        output[field] = value
  except:
    pass
  return output

# Function will be called once page has been requested and stored as a soup object. 
page = requests.get(test_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Output dictionary of patent classifications scraper. 
test_output = patent_classifications_scraper(soup)
test_output

In [ ]:
# Function will be called once page has been requested and stored as a soup object. 
page = requests.get(test_url)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
# Output dictionary of patent classifications scraper. 
test_output = patent_classifications_scraper(soup)

In [ ]:
test_output

{'A': 'HUMAN NECESSITIES',
 'A24': "TOBACCO; CIGARS; CIGARETTES; SIMULATED SMOKING DEVICES; SMOKERS' REQUISITES",
 'A24B': 'MANUFACTURE OR PREPARATION OF TOBACCO FOR SMOKING OR CHEWING; TOBACCO; SNUFF',
 'A24B15/00': 'Chemical features or treatment of tobacco; Tobacco substitutes, e.g. in liquid form',
 'A24B15/10': 'Chemical features of tobacco products or tobacco substitutes',
 'A24B15/16': 'Chemical features of tobacco products or tobacco substitutes of tobacco substitutes',
 'A24B15/167': 'Chemical features of tobacco products or tobacco substitutes of tobacco substitutes in liquid or vaporisable form, e.g. liquid compositions for electronic cigarettes',
 'A24F': "SMOKERS' REQUISITES; MATCH BOXES; SIMULATED SMOKING DEVICES",
 'A24F40/00': 'Electrically operated smoking devices; Component parts thereof; Manufacture thereof; Maintenance or testing thereof; Charging means specially adapted therefor',
 'A24F40/10': 'Devices using liquid inhalable precursors',
 'A24F47/00': "Smokers' re

In [ ]:
def create_classification_mapping(data : DataFrame) -> DataFrame:
  """ 
  Function to create a pivot table from the overall patent data
  """
  return data[["id", "classification_codes_broad"]].explode("classification_code_broad")


### Patent Company Cleaning

In [ ]:
def assignee_cleaning(companies : list, df):
  pass

In [ ]:
companies = companies_df['Company'].tolist()

In [ ]:
companies

['Reynolds American',
 'British American Tobacco',
 'Philip Morris',
 'Altria',
 'Imperial Tobacco',
 'Japan Tobacco',
 'KT&G',
 'Lorillard',
 'Juul',
 'Njoy',
 'Innokin',
 'Joyetech',
 'Geekvape',
 'Puff Bar',
 'Smok',
 'Uwell',
 'RELX UK',
 'Kimree',
 'Kangertech',
 'Hangsen',
 'Smoore']

In [ ]:
patents_df = pd.read_csv("company_patents_with_extras.csv")
patents_df.insert(3, "assignee_clean", 0)
patents_df.head(10)

,id,title,assignee,assignee_clean,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,patent_abstracts
0,US-D743248-S,Display package,Altria Client Services Llc,0,"Scott A. Fath, Monika Cascone",15/03/2013,11/08/2014,17/11/2015,17/11/2015,https://patents.google.com/patent/USD743248S1/en,https://patentimages.storage.googleapis.com/14...,NaN
1,US-2015313275-A1,Liquid aerosol formulation of an electronic sm...,Altria Client Services Llc,0,"Adam N. Anderson, Pauline Marcq, Gerd Kobal, M...",30/04/2014,21/01/2015,05/11/2015,NaN,https://patents.google.com/patent/US2015031327...,https://patentimages.storage.googleapis.com/a9...,A liquid aerosol formulation for an electronic...
2,US-D734145-S,Display package,Altria Client Services Llc,0,"Scott A. Fath, William J. Bogdziewicz, Robert ...",11/08/2014,11/08/2014,14/07/2015,14/07/2015,https://patents.google.com/patent/USD734145S1/en,https://patentimages.storage.googleapis.com/b9...,NaN
3,US-2016255876-A1,E-vaping device,Altria Client Services Llc,0,Ali A. Rostami,04/03/2015,04/03/2015,08/09/2016,NaN,https://patents.google.com/patent/US2016025587...,https://patentimages.storage.googleapis.com/4c...,An e-vaping device includes a first cartomizer...
4,US-2014238423-A1,Electronic smoking article,Altria Client Services Llc,0,"Christopher S. Tucker, Geoffrey Brandon Jordan",22/02/2013,20/02/2014,28/08/2014,NaN,https://patents.google.com/patent/US2014023842...,https://patentimages.storage.googleapis.com/1f...,An electronic smoking article includes a liqui...
5,US-9326547-B2,Electronic vaping article,Altria Client Services Llc,0,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,14/01/2013,03/05/2016,03/05/2016,https://patents.google.com/patent/US9326547B2/en,https://patentimages.storage.googleapis.com/a7...,An electronic smoking article includes an oute...
6,US-9282772-B2,Electronic vaping device,Altria Client Services Llc,0,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,14/01/2013,15/03/2016,15/03/2016,https://patents.google.com/patent/US9282772B2/en,https://patentimages.storage.googleapis.com/59...,An electronic smoking article includes a liqui...
7,US-9004073-B2,Electronic cigarette,Altria Client Services Llc,0,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,31/01/2013,14/04/2015,14/04/2015,https://patents.google.com/patent/US9004073B2/en,https://patentimages.storage.googleapis.com/47...,An electronic smoking article includes a liqui...
8,US-9510623-B2,Electronic cigarette,Altria Client Services Llc,0,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,31/01/2013,06/12/2016,06/12/2016,https://patents.google.com/patent/US9510623B2/en,https://patentimages.storage.googleapis.com/f3...,An electronic cigarette includes a liquid supp...
9,US-8997754-B2,Electronic cigarette,Altria Client Services Llc,0,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,31/01/2013,07/04/2015,07/04/2015,https://patents.google.com/patent/US8997754B2/en,https://patentimages.storage.googleapis.com/15...,An electronic smoking article includes a liqui...


In [ ]:
for company in companies:
  patents_df.loc[patents_df['assignee'].str.contains(company, na=False), 'assignee_clean'] = company
else:
  patents_df['assignee_clean'] = patents_df.apply(lambda x: x['assignee'] if x['assignee_clean'] == 0 else x['assignee_clean'], axis = 1)

In [ ]:
patents_df['assignee_clean'].value_counts()

Philip Morris                                                                3578
Japan Tobacco                                                                1427
Altria                                                                       1364
British American Tobacco                                                      801
R.J. Reynolds Tobacco Company                                                 683
                                                                             ... 
Mycogen Corporation                                                             1
Yoshiyuki Yamada, Masafumi Tarora, Atsushi Nagao                                1
Bray Andrew J, Tearle Alan D, Steven Holford                                    1
Charles Dendy, Philip DIANA, Zack WEIGENSBERG, Kai Cao, Patrick J. Cobler       1
Firooz Rasouli, Clive Scorey, Mckernan John, Cunningham John F.                 1
Name: assignee_clean, Length: 1082, dtype: int64

In [ ]:
patents_df.head(10)

,id,title,assignee,assignee_clean,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,patent_abstracts
0,US-D743248-S,Display package,Altria Client Services Llc,Altria,"Scott A. Fath, Monika Cascone",15/03/2013,11/08/2014,17/11/2015,17/11/2015,https://patents.google.com/patent/USD743248S1/en,https://patentimages.storage.googleapis.com/14...,NaN
1,US-2015313275-A1,Liquid aerosol formulation of an electronic sm...,Altria Client Services Llc,Altria,"Adam N. Anderson, Pauline Marcq, Gerd Kobal, M...",30/04/2014,21/01/2015,05/11/2015,NaN,https://patents.google.com/patent/US2015031327...,https://patentimages.storage.googleapis.com/a9...,A liquid aerosol formulation for an electronic...
2,US-D734145-S,Display package,Altria Client Services Llc,Altria,"Scott A. Fath, William J. Bogdziewicz, Robert ...",11/08/2014,11/08/2014,14/07/2015,14/07/2015,https://patents.google.com/patent/USD734145S1/en,https://patentimages.storage.googleapis.com/b9...,NaN
3,US-2016255876-A1,E-vaping device,Altria Client Services Llc,Altria,Ali A. Rostami,04/03/2015,04/03/2015,08/09/2016,NaN,https://patents.google.com/patent/US2016025587...,https://patentimages.storage.googleapis.com/4c...,An e-vaping device includes a first cartomizer...
4,US-2014238423-A1,Electronic smoking article,Altria Client Services Llc,Altria,"Christopher S. Tucker, Geoffrey Brandon Jordan",22/02/2013,20/02/2014,28/08/2014,NaN,https://patents.google.com/patent/US2014023842...,https://patentimages.storage.googleapis.com/1f...,An electronic smoking article includes a liqui...
5,US-9326547-B2,Electronic vaping article,Altria Client Services Llc,Altria,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,14/01/2013,03/05/2016,03/05/2016,https://patents.google.com/patent/US9326547B2/en,https://patentimages.storage.googleapis.com/a7...,An electronic smoking article includes an oute...
6,US-9282772-B2,Electronic vaping device,Altria Client Services Llc,Altria,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,14/01/2013,15/03/2016,15/03/2016,https://patents.google.com/patent/US9282772B2/en,https://patentimages.storage.googleapis.com/59...,An electronic smoking article includes a liqui...
7,US-9004073-B2,Electronic cigarette,Altria Client Services Llc,Altria,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,31/01/2013,14/04/2015,14/04/2015,https://patents.google.com/patent/US9004073B2/en,https://patentimages.storage.googleapis.com/47...,An electronic smoking article includes a liqui...
8,US-9510623-B2,Electronic cigarette,Altria Client Services Llc,Altria,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,31/01/2013,06/12/2016,06/12/2016,https://patents.google.com/patent/US9510623B2/en,https://patentimages.storage.googleapis.com/f3...,An electronic cigarette includes a liquid supp...
9,US-8997754-B2,Electronic cigarette,Altria Client Services Llc,Altria,"Christopher S. Tucker, Geoffrey Brandon Jordan...",31/01/2012,31/01/2013,07/04/2015,07/04/2015,https://patents.google.com/patent/US8997754B2/en,https://patentimages.storage.googleapis.com/15...,An electronic smoking article includes a liqui...


In [ ]:
patents_df.to_csv("company_patents_with_extras_v2.csv")